In [1]:
import datetime as dt
import holidays
import requests as req
import pandas as pd
import yfinance as yf
import sqlite3 as sqlite

nan = float('nan')

In [ ]:
def GET_INDICE_COMPONENTS():
    res  = req.get('https://www.slickcharts.com/sp500', headers={ 'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36', 'X-Requested-With':'XMLHttpRequest' })
    page = pd.read_html(res.content)
    tbl  = page[0]
    return tbl

SNP_500 = GET_INDICE_COMPONENTS()['Symbol']

In [3]:
ETFS        = ['SPY','DIA','QQQ','XLF','XLC','XLE','XLU','XLP','XLK','XLV','XLI','XLY','XLB','XOP','XBI','XRT','XHB','XME','XLRE','IWM','IYR','NOBL','AMLP','OIH','KRE','VNQ','ITB','KBE','IBB','SMH']
TICKERS     = [*ETFS, *SNP_500.head(50)]
TODAY       = dt.date.today()
IS_HOLIDAY  = TODAY in holidays.US(years=TODAY.year)

In [4]:
if IS_HOLIDAY: raise Exception('IS_HOLIDAY')

In [ ]:
def GET_YF_OPT_CHAINS(TICKERS):

    # ================ Helpers ================ #
    def _yf_Ticker(*a,**b):
        try:    return yf.Ticker(*a,**b)
        except: return None

    def _yf_getPrice(Asset, alt=nan):
        try:    return Asset.info.get('previousClose')
        except: return alt

    def _yf_getExps(Asset, alt=[]):
        try:    return Asset.options
        except: return alt
    
    
    # ================ Main ================ #
    Pipe = []
    for TICKER in TICKERS:
        Asset = _yf_Ticker(TICKER)
        price = _yf_getPrice(Asset)

        for EXP in _yf_getExps(Asset):
            for OPT in ['calls','puts']:
                try: 
                    chain = getattr(Asset.option_chain(EXP), OPT)
                    Pipe.append(pd.DataFrame({
                        'Date':     TODAY, 
                        'Ticker':   TICKER, 
                        'Price':    price, 
                        'Code':     chain['contractSymbol'],
                        'Opt':      OPT[0].capitalize(), 
                        'Expiry':   EXP, 
                        'Strike':   chain['strike'], 
                        'Vol':      chain['volume'], 
                        'OI':       chain['openInterest'], 
                        'Ask':      chain['ask'], 
                        'Bid':      chain['bid'], 
                        'Last':     chain['lastPrice'], 
                    }))
                except Exception as E: print(E)
            pass
        pass
    pass
    return pd.concat(Pipe, ignore_index=1)

Chains = GET_YF_OPT_CHAINS(TICKERS)
# Chains

In [6]:
con = sqlite.connect('Filebase.db')
Chains.to_sql('Chain', con, if_exists='append', index=0)

117671